# CEQR API Test

Search CEQR (City Environmental Quality Review) projects by borough, block, and lot.

**No browser needed** - fully automated with Python.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

print("✅ Ready")


## Search Function


In [ ]:
def search_ceqr(borough, block, lot=""):
    """
    Search CEQR database by borough, block, and lot.
    
    Returns: tuple (success: bool, response or error message)
    """
    url = "https://a002-ceqraccess.nyc.gov/ceqr/"
    session = requests.Session()
    
    # Step 1: GET initial page to get VIEWSTATE
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
    }
    
    try:
        print(f"🔍 Searching: {borough}, Block {block}" + (f", Lot {lot}" if lot else ""))
        
        # GET the page
        init_resp = session.get(url, headers=headers, timeout=30)
        if init_resp.status_code != 200:
            return False, f"Failed to load page: {init_resp.status_code}"
        
        # Extract VIEWSTATE fields
        soup = BeautifulSoup(init_resp.text, 'html.parser')
        viewstate = soup.find('input', {'id': '__VIEWSTATE'})
        viewstate_gen = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})
        eventval = soup.find('input', {'id': '__EVENTVALIDATION'})
        
        if not viewstate:
            return False, "Could not find VIEWSTATE"
        
        print("✅ Got session")
        
        # Step 2: POST search
        form_data = {
            "__LASTFOCUS": "",
            "__EVENTTARGET": "",
            "__EVENTARGUMENT": "",
            "__VIEWSTATE": viewstate['value'],
            "__VIEWSTATEGENERATOR": viewstate_gen['value'] if viewstate_gen else "F2CE38DF",
            "__SCROLLPOSITIONX": "0",
            "__SCROLLPOSITIONY": "0",
            "__VIEWSTATEENCRYPTED": "",
            "__EVENTVALIDATION": eventval['value'] if eventval else "",
            "ctl00$MainContent$txtKeyword": "",
            "ctl00$MainContent$ddlLeadAgency": "XYU@2!",
            "ctl00$MainContent$txtCeqrNumber": "",
            "ctl00$MainContent$txtProjectName": "",
            "ctl00$MainContent$ddlCommunityDistrict": "XYU@2!",
            "ctl00$MainContent$ddlBorough": borough,
            "ctl00$MainContent$txtBlock": block,
            "ctl00$MainContent$txtLot": lot,
            "ctl00$MainContent$btnSearch": " Search"
        }
        
        post_headers = {
            **headers,
            "Content-Type": "application/x-www-form-urlencoded",
            "Origin": "https://a002-ceqraccess.nyc.gov",
            "Referer": url,
            "Cache-Control": "max-age=0"
        }
        
        response = session.post(url, headers=post_headers, data=form_data, timeout=30)
        
        if response.status_code != 200:
            return False, f"Search failed: {response.status_code}"
        
        # Check for results
        if 'grdSearchResults' in response.text or 'Search Results' in response.text:
            print("✅ Got results")
            return True, response
        elif 'Error' in response.text or 'Unhandled' in response.text:
            return False, "Server error"
        else:
            print("⚠️  No results found")
            return True, response
            
    except Exception as e:
        return False, f"Error: {str(e)}"

print("✅ Function loaded")


## Parse Results


In [ ]:
def parse_ceqr_results(response):
    """Extract CEQR results table from HTML response."""
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find results table
    table = soup.find('table', {'id': lambda x: x and 'grdSearchResults' in x})
    
    if not table:
        # Try finding by content
        tables = soup.find_all('table')
        for t in tables:
            if 'CEQR Number' in t.get_text() or 'Project Name' in t.get_text():
                table = t
                break
    
    if not table:
        print("⚠️  No results table found")
        return None
    
    # Extract rows
    rows = table.find_all('tr')
    if not rows:
        return None
    
    # Get headers
    headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
    
    # Get data
    data = []
    for row in rows[1:]:
        cells = row.find_all(['td', 'th'])
        if cells:
            row_data = [cell.get_text(strip=True) for cell in cells]
            if any(cell.strip() for cell in row_data):
                data.append(row_data)
    
    if not data:
        return None
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers[:len(data[0])])
    return df

print("✅ Parser loaded")


## Test Search


In [ ]:
# Test with Brooklyn Block 7061 (from your curl example)
success, result = search_ceqr("Brooklyn", "7061")

if success:
    df = parse_ceqr_results(result)
    if df is not None:
        print(f"\n📊 Found {len(df)} results\n")
        print("=" * 100)
        print(df.to_string(index=False))
        print("=" * 100)
    else:
        print("No results to display")
else:
    print(f"❌ Error: {result}")
